In [2]:
# Ensure the necessary packages are installed
!pip install transformers[torch]
!pip install accelerate -U

# Restart your kernel after running the above commands

import urllib.request
import zipfile
import os
from pathlib import Path
import pandas as pd

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

def download_and_unzip(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return

    # Downloading the file
    with urllib.request.urlopen(url) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())

    # Unzipping the file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    # Add .tsv file extension
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as {data_file_path}")

download_and_unzip(url, zip_path, extracted_path, data_file_path)

df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df["Label"] = df["Label"].map({"ham": 0, "spam": 1})

def create_balanced_dataset(df):
    num_spam = df[df["Label"] == "spam"].shape[0]
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

# Assuming the ScamDataNew.csv and scams13.xlsx files are already in place
# data = pd.read_csv('/path/to/ScamDataNew.csv')
# testData = pd.read_excel('/path/to/scams13.xlsx')

# For demonstration purposes, I'll create dummy data to replace the above two datasets
data = pd.DataFrame({
    'Label': [0, 1],
    'Scammer': ["This is a ham message.", "This is a spam message."]
})
testData = pd.DataFrame({
    'Label': [0, 1],
    'Scammer': ["Another ham message.", "Another spam message."]
})

balanced_df = balanced_df.rename(columns={'Text': 'Scammer'})
balanced_df = pd.concat([balanced_df, data, testData], ignore_index=True)

def random_split(df, train_frac, validation_frac):
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset



sms_spam_collection/SMSSpamCollection.tsv already exists. Skipping download and extraction.
Series([], Name: count, dtype: int64)


In [3]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from torch.utils.data import Dataset
import pandas as pd

# Add a padding token to the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None):
        self.data = pd.read_csv(csv_file)
        print(f"Loaded data from {csv_file}:")
        print(self.data.head())  # Print the first few rows for debugging

        self.tokenizer = tokenizer
        self.max_length = max_length or 1000
        self.encoded_texts = self.tokenizer(
            self.data["Scammer"].tolist(),
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        print(f"Tokenization successful for {csv_file}. Sample encoding:")
        print(self.encoded_texts)

    def __getitem__(self, index):
        item = {key: val[index] for key, val in self.encoded_texts.items()}
        item['labels'] = torch.tensor(self.data.iloc[index]["Label"], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.data)

# Load datasets
print("Loading training dataset...")
train_dataset = SpamDataset('train.csv', tokenizer)

print("Loading validation dataset...")
validation_dataset = SpamDataset('test.csv', tokenizer)

# Model setup
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    evaluation_strategy="epoch",
    save_steps=10_000,
    save_total_limit=2,
)

# Create a data collator
data_collator = DataCollatorWithPadding(tokenizer)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
)

# Train the model
# trainer.train()


Loading training dataset...
Loaded data from train.csv:
   Label                 Scammer
0      1   Another spam message.
1      0  This is a ham message.
Tokenization successful for train.csv. Sample encoding:
{'input_ids': tensor([[ 6610, 18084,  3275,  ..., 50256, 50256, 50256],
        [ 1212,   318,   257,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
Loading validation dataset...
Loaded data from test.csv:
   Label                  Scammer
0      1  This is a spam message.
1      0     Another ham message.
Tokenization successful for test.csv. Sample encoding:
{'input_ids': tensor([[ 1212,   318,   257,  ..., 50256, 50256, 50256],
        [ 6610,  8891,  3275,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.762051
2,No log,0.782540


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from transformers import GPT2Model, GPT2ForSequenceClassification

# Define student model (a smaller version of GPT-2)
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.classifier = nn.Linear(self.gpt2.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, -1, :])
        return logits

student_model = StudentModel()

# Loss function for distillation
def distillation_loss(student_logits, teacher_logits, temperature=2.0):
    soft_targets = nn.functional.softmax(teacher_logits / temperature, dim=1)
    soft_predictions = nn.functional.log_softmax(student_logits / temperature, dim=1)
    loss = nn.functional.kl_div(soft_predictions, soft_targets, reduction='batchmean') * (temperature ** 2)
    return loss

# Training loop for distillation
def train_student_model(student_model, teacher_model, train_loader, epochs=3, lr=3e-2, temperature=2.0):
    optimizer = optim.Adam(student_model.parameters(), lr=lr)
    teacher_model.eval()  # Set teacher model to evaluation mode

    for epoch in range(epochs):
        student_model.train()  # Set student model to training mode
        total_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']

            # Forward pass through teacher model
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits

            # Forward pass through student model
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

            # Compute distillation loss
            loss = distillation_loss(student_logits, teacher_logits, temperature)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Create DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=data_collator)

# Train the student model using distillation
train_student_model(student_model, model, train_loader)


In [ ]:
student_model.save_pretrained('./distilled_student_model')
tokenizer.save_pretrained('./distilled_student_model')
